SOP0124 - List Keys For Encryption At Rest.
===========================================

Description
-----------

Use this notebook to connect to the `controller` database to list all
keys which are used by the encryption at rest feature of SQL Server Big
Data Cluster. These keys are used by HDFS encryption zone.

Sample output: Name, Cipher, Bit Length, Master Key Version mykey1,
AES/CTR/NoPadding, 256, hdfs-encryption-master-key@0 mykey2,
AES/CTR/NoPadding, 128, hdfs-encryption-master-key@0 mykey3,
AES/CTR/NoPadding, 192, hdfs-encryption-master-key@0 securelakekey,
AES/CTR/NoPadding, 256, hdfs-encryption-master-key@0

Steps
-----

### Instantiate Kubernetes client

In [ ]:
# Instantiate the Python Kubernetes client into 'api' variable

import os
from IPython.display import Markdown

try:
    from kubernetes import client, config
    from kubernetes.stream import stream
except ImportError: 

    # Install the Kubernetes module
    import sys
    !{sys.executable} -m pip install kubernetes    
    
    try:
        from kubernetes import client, config
        from kubernetes.stream import stream
    except ImportError:
        display(Markdown(f'HINT: Use [SOP059 - Install Kubernetes Python module](../install/sop059-install-kubernetes-module.ipynb) to resolve this issue.'))
        raise

if "KUBERNETES_SERVICE_PORT" in os.environ and "KUBERNETES_SERVICE_HOST" in os.environ:
    config.load_incluster_config()
else:
    try:
        config.load_kube_config()
    except:
        display(Markdown(f'HINT: Use [TSG118 - Configure Kubernetes config](../repair/tsg118-configure-kube-config.ipynb) to resolve this issue.'))
        raise

api = client.CoreV1Api()

print('Kubernetes client instantiated')

### Get the namespace for the big data cluster

Get the namespace of the Big Data Cluster from the Kuberenetes API.

**NOTE:**

If there is more than one Big Data Cluster in the target Kubernetes
cluster, then either:

-   set \[0\] to the correct value for the big data cluster.
-   set the environment variable AZDATA\_NAMESPACE, before starting
    Azure Data Studio.

In [ ]:
# Place Kubernetes namespace name for BDC into 'namespace' variable

if "AZDATA_NAMESPACE" in os.environ:
    namespace = os.environ["AZDATA_NAMESPACE"]
else:
    try:
        namespace = api.list_namespace(label_selector='MSSQL_CLUSTER').items[0].metadata.name
    except IndexError:
        from IPython.display import Markdown
        display(Markdown(f'HINT: Use [TSG081 - Get namespaces (Kubernetes)](../monitor-k8s/tsg081-get-kubernetes-namespaces.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [TSG010 - Get configuration contexts](../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [SOP011 - Set kubernetes configuration context](../common/sop011-set-kubernetes-context.ipynb) to resolve this issue.'))
        raise

print('The kubernetes namespace for your big data cluster is: ' + namespace)

### Python function queries `controller` database and return results.

In [ ]:
try:
  import pandas
except ModuleNotFoundError:
  !{sys.executable} -m pip install --user pandas
  import pandas
from io import StringIO
pandas.set_option('display.max_colwidth', -1)
name = 'controldb-0'
container = 'mssql-server'

def get_dataframe(query):
    command=f"""export SQLCMDPASSWORD=$(cat /var/run/secrets/credentials/mssql-sa-password/password);
    /opt/mssql-tools/bin/sqlcmd -b -S . -U sa -Q "SET NOCOUNT ON;
    {query}" -d controller  -s"^" -W > /tmp/out.csv;
    sed -i 2d /tmp/out.csv; cat /tmp/out.csv"""
    output=stream(api.connect_get_namespaced_pod_exec, name, namespace, command=['/bin/sh', '-c', command], container=container, stderr=True, stdout=True)
    return pandas.read_csv(StringIO(output), sep='^')

print("Function 'get_dataframe' defined")

### List all keys for Hadoop encryption at rest.

In [ ]:
import json
import base64

df = get_dataframe("select account_name, CAST(application_metadata AS NVARCHAR(4000)) from Credentials where ISJSON(application_metadata) > 0 and JSON_VALUE(application_metadata, '$.vaultname') in ('hdfsvault-svc', 'mssqlvault-svc') and type = '2'")

print("{:<20} {:<25} {:<20} {:<30}".format('Name', 'Cipher', 'Bit Length', 'Main Key Version'))
print("-------------------------------------------------------------------------------------------------")

for index, row in df.iterrows():
    key_name = row[0]
    applicationJson = json.loads(row[1])
    cipher = applicationJson["cipher"]
    bit_length = applicationJson["bitLength"]
    originalUrl = base64.b64decode(applicationJson["master_key_version_name"]).decode()
    splitItems = originalUrl.split("/")
    master_key_version = splitItems[-2] + "@" + splitItems[-1]
    print("{:<20} {:<25} {:<20} {:<30}".format(key_name, cipher, bit_length, master_key_version))

df = get_dataframe("select account_name, application_metadata from Credentials where ISJSON(application_metadata) > 0 and JSON_VALUE(application_metadata, '$.vaultname') in ('hdfsvault-svc', 'mssqlvault-svc') and type = '3'")
print("Encryption Key protector")
print("-------------------------")
for index, row in df.iterrows():
    key_name = row[0]
    print(key_name)

In [ ]:
print("Notebook execution is complete.")